In [84]:
from collections import defaultdict
from collections import Counter
from ete3 import PhyloTree
import numpy as np
from tqdm import tqdm_notebook
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import time
import csv
import copy

Load gene families

In [11]:
Fams={}
with open('Orthogroups_dino.csv') as csvfile:
    species=[]
    for i,rc in enumerate(csv.reader(csvfile, delimiter='\t', quotechar='"')):
        if i==0:
            species=[sp.split('_')[0] for sp in rc[1:]]
            continue
        genBySp=dict((species[j],[g.split('|')[1] for g in spg.split(',')]) for j,spg in enumerate(rc[1:]) if not spg=='')
        #genBySp=
        Fams[rc[0]]=genBySp
        #if i==5: break


Creates non-redundant species set

In [12]:
spSet=set()
for fid,gbs in Fams.items():
    for sp in gbs.keys():
        spSet.add(sp)
print(spSet)


{'mli', 'dme', 'sma', 'cin', 'sme', 'ava', 'dgy', 'lan', 'hsa', 'nge', 'ppa', 'sko', 'hro', 'hel', 'aqu', 'pye', 'tca', 'loc', 'obi', 'cel', 'nve', 'pau', 'cgi', 'mle', 'odi', 'lgi', 'cte', 'bla', 'rva'}


Load species tree

In [13]:
spT=PhyloTree(open('tree_dino.tre').read(), format=1)

## Testing family expansions

In [53]:
#only testing species expansion
def fishExp(gbs,tbs):
    cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
    med=np.median(list(cbs.values()))
    #print(cbs,med)
    spval={}
    for sp,st in cbs.items():
        nis=sum(cbs.values())-st
        nit=sum(tbs.values())-tbs[sp]-sum(cbs.values())+st
        odds,pval=stats.fisher_exact([[st, tbs[sp]-st], [nis, nit]],alternative='greater')
        nmed=np.median([cbs[ssp] for ssp in cbs if not ssp==sp])
        tmed=np.median([tbs[ssp] for ssp in tbs if not ssp==sp])
        m_odds,m_pval=stats.fisher_exact([[st, tbs[sp]-st], [nmed, tmed]],alternative='greater')
        #print(sp,pval,[st, tbs[sp]-st], [nis, nit],m_pval,[st, tbs[sp]-st], [nmed, tmed])
        spval[sp]=(cbs[sp],m_pval)

    return spval

calculate number of genes per species

In [15]:
tbs=defaultdict(int)
for fid,gbs in Fams.items():
    for sp in gbs:
        tbs[sp]+=len(gbs[sp])

In [54]:
toosmall=0
expRes=[]
for fid,gbs in tqdm_notebook(Fams.items()):
    cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
    if len(cbs.keys())<3 or sum(cbs.values())<5:
        toosmall+=1
        continue
    try:
        spval=fishExp(gbs,tbs)
        expRes.append((fid,spval))
    except:
        print(fid,cbs)

In [59]:
print(len(Fams),len(expRes),toosmall)

22544 11656 10888


In [63]:
spFam,spPval=defaultdict(list),defaultdict(list)
for fid,spval in expRes:
    for sp in spval:
        ct,pval=spval[sp]
        spFam[sp].append(fid)                     
        spPval[sp].append(pval)

In [74]:
famEnrich=defaultdict(dict)
for sp in spPval:
    #signif_pvals = sidak(spPval[sp], alpha=0.05)
    #before_adj=['True' if p < 0.05  else 'False' for p in spPval[sp] ]
    #1signif_pvals = lsu(np.array(spPval[sp]), q=0.05) #this is benferroni-hochberg
    adj=multipletests(pvals=np.array(spPval[sp]), alpha=0.05, method="fdr_bh")
    print (sp,Counter(adj[0]))
    for fam,pth,pval in zip(spFam[sp],adj[0],adj[1]):
        famEnrich[fam][sp]=(pth,pval)
        


aqu Counter({False: 6404, True: 135})
ava Counter({False: 6291, True: 24})
bla Counter({False: 7902, True: 41})
cel Counter({False: 4947, True: 40})
cgi Counter({False: 7977, True: 36})
cin Counter({False: 6422, True: 16})
cte Counter({False: 8599, True: 74})
hel Counter({False: 5663, True: 38})
hro Counter({False: 6455, True: 46})
hsa Counter({False: 7474, True: 22})
lan Counter({False: 9040, True: 10})
lgi Counter({False: 8504, True: 16})
loc Counter({False: 7344, True: 11})
mle Counter({False: 5318, True: 36})
mli Counter({False: 5969, True: 14})
nge Counter({False: 8804, True: 63})
nve Counter({False: 7388, True: 41})
obi Counter({False: 7966, True: 47})
odi Counter({False: 4965, True: 29})
pau Counter({False: 8456, True: 16})
ppa Counter({False: 4569, True: 64})
pye Counter({False: 8915, True: 27})
rva Counter({False: 5333, True: 54})
sko Counter({False: 8430, True: 63})
sma Counter({False: 6108, True: 40})
tca Counter({False: 6164, True: 39})
dgy Counter({False: 6983, True: 8})
d

In [ ]:
with open('Orthogroups_enrich.txt','w') as out:
    for fid,gbs in Fams.items():
        cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
        pvl=famEnrich[fid]
        enriched=set([])
        outList=[fid,]
        out.write('\t'.join(outList))
    

## Gain and losses

In [81]:
def checkFam(fam,spTd):
    fid,gbs=fam
    spset=list(gbs.keys())
    if len(spset)>1:
        phtyp=spT.get_common_ancestor(list(gbs.keys()))
    else:
        phtyp=[l for l in spT.get_leaves() if l.name==spset[0]][0]
    #print phtyp.name
    #phyloCt[phtyp.name]+=1
    #gbs[Ai]
    ndesc=len([l.name for l in phtyp.get_leaves()])
    for leaf in spTd:
        pv='1' if leaf.name in gbs else '0'
        leaf.add_features(presence=pv)
    lost=[node.name  for node in phtyp.get_monophyletic(values=['0'], target_attr="presence")]
    #print spTd.get_ascii(attributes=["name","presence"], show_internal=True)
    #for node in spTd.get_monophyletic(values=['0'], target_attr="presence"):
    #    print node.name
    #    print node.get_ascii(attributes=["presence", "name"], show_internal=False)
    return fid,len(gbs.keys()),ndesc,phtyp.name,lost



In [99]:
origins=defaultdict(int)
losses=defaultdict(int)
idGainLoss={}
for fam in Fams.items():
    fid,nspec,ndesc,oritax,lost=checkFam(fam,spT)
    idGainLoss[fid]=(nspec,ndesc,oritax,lost)
    origins[oritax]+=1
    for tax in lost:
        losses[tax]+=1

In [96]:
spTi=copy.deepcopy(spT)
for node in spTi.traverse("postorder"):
    #print node.name,origins[node.name],losses[node.name]
    #node.add_features(origins=origins[node.name],losses=losses[node.name])
    node.name="{0}_{1}_{2}]".format(node.name,origins[node.name],losses[node.name])

In [97]:
print(spTi.write(format=7))
#print(spTi.get_ascii(attributes=["name"], show_internal=True))

(aqu_312_0_:1,(mle_101_2483_:1,(nve_61_1769_:1,((sko_72_1625_:1,(bla_35_1584_:1,((cin_31_735_:1,odi_102_2203_:1)urochordata_60_1542_,(loc_8_462_:1,hsa_16_350_:1)vertebrata_795_909_)olfactores_50_992_)chordata_66_758_)deuterostomes_131_1270_,(((cel_133_534_:1,ppa_85_788_:1)nematodes_1546_2926_,(rva_139_2419_:1,(sma_42_1268_:1,(dme_39_767_:1,tca_48_344_:1)insects_385_880_)mandibulata_90_462_)panarthropoda_29_601_)ecdysozoa_37_3652_,(ava_509_5498_:1,((mli_678_781_:1,sme_66_2587_:1)platyhelminthes_72_5273_,((obi_52_2610_:1,(lgi_70_1773_:1,(cgi_75_1617_:1,pye_128_769_:1)bivalvia_422_647_)moll_cl_184_483_)mollusca_209_2141_,((dgy_45_3813_:1,(hel_96_4605_:1,(cte_121_610_:1,hro_76_3281_:1)anne_cl1_50_872_)anne_cl2_148_296_)annelids_153_1848_,(nge_240_1836_:1,(pau_76_1481_:1,lan_118_710_:1)lophophorata_76_1227_)spir_cl_100_1399_)spir_cl1_357_579_)spir_cl2_900_230_)spir_cl3_396_276_)spiralia_227_211_)protostomia_729_215_)bilateria_2067_96_)parahoxozoa_1681_20_)eumetazoa_993_0_);


## Output

In [117]:
with open('Orthogroups_stats.tsv','w') as out:
    out.write("FID\tnb_genes\tnb_species\torigin\tlost_sp\tlost_taxa\texpanded_tax\n")
    for fid,gbs in Fams.items():
        nspec,ndesc,oritax,lost=idGainLoss[fid]
        ngn=sum([len(gl) for gl in gbs.values()])
        exp=[s for s,p in famEnrich[fid].items() if p[0]]
        #rpvals=zip(pvals.keys(),[round(-np.log10(l[1]),2) for l in pvals.values()])
        outL=[fid,str(ngn),str(nspec),oritax,str(ndesc-nspec),','.join(lost),','.join(exp)]
        out.write('\t'.join(outL)+'\n')